# Session 5: Segregation & Moving Boundary Coupling

**Coupled Oxidation-Diffusion with Dopant Segregation**

This notebook demonstrates:
- Dopant segregation at Si/SiO₂ interface
- Moving boundary tracking during oxidation
- Pile-up effects for low-k dopants (Arsenic)
- Depletion effects for moderate-k dopants (Boron)
- Mass conservation in coupled problems
- Interface velocity and consumption

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import sys
from pathlib import Path

# Add module to path
sys.path.insert(0, str(Path.cwd().parent))

from core.segregation import (
    SegregationModel,
    MovingBoundaryTracker,
    arsenic_pile_up_demo,
    boron_depletion_demo,
    SEGREGATION_COEFFICIENTS
)
from core.fick_fd import Fick1D
from core.erfc import diffusivity

# Plot styling
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11

print("✓ Imports successful")
print("Session 5: Segregation & Moving Boundary Coupling")
print(f"\nSegregation Coefficients:")
for dopant, k in SEGREGATION_COEFFICIENTS.items():
    print(f"  {dopant.capitalize()}: k = {k}")

## 1. Segregation Physics Fundamentals

When silicon oxidizes, dopants partition between oxide and silicon:

$$k = \frac{C_{\text{oxide}}}{C_{\text{silicon}}}$$

**Key Effects:**
- **k < 1**: Dopants rejected from oxide → pile-up in Si
- **k > 1**: Dopants prefer oxide → depletion in Si
- **k ≈ 1**: No segregation effect

**Typical Values:**
- Arsenic: k ≈ 0.02 (strong pile-up)
- Phosphorus: k ≈ 0.1 (moderate pile-up)
- Boron: k ≈ 0.3 (mild pile-up)

In [ ]:
# Visualize segregation coefficients
dopants = list(SEGREGATION_COEFFICIENTS.keys())
k_values = list(SEGREGATION_COEFFICIENTS.values())

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart of k values
colors = ['red', 'green', 'blue', 'orange']
ax1.bar(dopants, k_values, color=colors, alpha=0.7)
ax1.axhline(1.0, color='k', linestyle='--', alpha=0.5, label='k = 1 (no segregation)')
ax1.set_ylabel('Segregation Coefficient k')
ax1.set_title('Dopant Segregation Coefficients\n(Si/SiO₂ Interface)')
ax1.legend()
ax1.grid(True, alpha=0.3, axis='y')

# Pile-up strength (1-k)
pile_up_strength = [1 - k for k in k_values]
ax2.bar(dopants, pile_up_strength, color=colors, alpha=0.7)
ax2.set_ylabel('Rejection Fraction (1 - k)')
ax2.set_title('Dopant Pile-Up Strength\nHigher = Stronger Pile-Up')
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("Lower k → Stronger rejection → More pile-up")
print(f"Arsenic (k={SEGREGATION_COEFFICIENTS['arsenic']}) has strongest pile-up")

## 2. Arsenic Pile-Up During Dry Oxidation

Arsenic has k ≈ 0.02, causing strong dopant rejection and pile-up.

**Physical Process:**
1. Oxide grows at Si surface
2. Si/SiO₂ interface moves inward
3. Arsenic rejected from oxide (98% stays in Si)
4. Dopant accumulates at moving interface
5. Concentration spike forms

In [ ]:
# Demonstrate arsenic pile-up
print("Simulating arsenic pile-up during dry oxidation...")
print("Conditions: 1000°C, 60 minutes")

x_as, C_as = arsenic_pile_up_demo(T=1000, t=60, C_initial=1e19)

# Plot results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Linear scale
ax1.plot(x_as, C_as, 'r-', linewidth=2, label='After oxidation')
ax1.axhline(1e19, color='b', linestyle='--', alpha=0.5, label='Initial uniform doping')
ax1.set_xlabel('Depth (nm)')
ax1.set_ylabel('Concentration (cm⁻³)')
ax1.set_title('Arsenic Pile-Up During Dry Oxidation\n1000°C, 60 min')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, 200)

# Log scale
ax2.semilogy(x_as, C_as, 'r-', linewidth=2, label='After oxidation')
ax2.axhline(1e19, color='b', linestyle='--', alpha=0.5, label='Initial')
ax2.set_xlabel('Depth (nm)')
ax2.set_ylabel('Concentration (cm⁻³)')
ax2.set_title('Arsenic Pile-Up (Log Scale)')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0, 200)
ax2.set_ylim(1e18, 1e20)

plt.tight_layout()
plt.show()

# Analyze pile-up
C_max = C_as.max()
C_initial = 1e19
pile_up_ratio = C_max / C_initial

print(f"\nResults:")
print(f"  Initial concentration: {C_initial:.2e} cm⁻³")
print(f"  Peak concentration: {C_max:.2e} cm⁻³")
print(f"  Pile-up ratio: {pile_up_ratio:.2f}x")
print(f"\n✓ Arsenic shows {(pile_up_ratio-1)*100:.0f}% concentration increase due to segregation")

## 3. Boron Behavior During Oxidation

Boron has k ≈ 0.3, showing moderate segregation effects.

**Differences from Arsenic:**
- Less rejection (70% stays in Si vs 98% for As)
- Weaker pile-up effect
- Enhanced diffusion in wet O₂ can cause depletion

In [ ]:
# Demonstrate boron behavior
print("Simulating boron during wet oxidation...")
print("Conditions: 1100°C, 120 minutes (wet O₂)")

x_b, C_b = boron_depletion_demo(T=1100, t=120, C_initial=1e18)

# Plot results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Concentration profile
ax1.plot(x_b, C_b, 'b-', linewidth=2, label='After oxidation')
ax1.axhline(1e18, color='g', linestyle='--', alpha=0.5, label='Initial uniform doping')
ax1.set_xlabel('Depth (nm)')
ax1.set_ylabel('Concentration (cm⁻³)')
ax1.set_title('Boron During Wet Oxidation\n1100°C, 120 min')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, 300)

# Normalized profile
C_b_norm = C_b / 1e18
ax2.plot(x_b, C_b_norm, 'b-', linewidth=2)
ax2.axhline(1.0, color='g', linestyle='--', alpha=0.5, label='Initial level')
ax2.set_xlabel('Depth (nm)')
ax2.set_ylabel('Normalized Concentration (C/C₀)')
ax2.set_title('Boron Profile (Normalized)')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0, 300)
ax2.set_ylim(0, 2)

plt.tight_layout()
plt.show()

print(f"\nBoron segregation coefficient: k = {SEGREGATION_COEFFICIENTS['boron']}")
print(f"Rejection: {(1-SEGREGATION_COEFFICIENTS['boron'])*100:.0f}% stays in Si")

## 4. Comparison: Different Dopants

Compare pile-up effects for different dopants under identical conditions.

In [ ]:
# Compare multiple dopants
print("Comparing dopants under identical conditions...")
print("Temperature: 1000°C, Time: 60 min, Initial: 1e18 cm⁻³")

# Setup
solver = Fick1D(x_max=400, dx=1.0, refine_surface=False)
x = solver.x
C0 = np.full(len(x), 1e18)

def oxide_model(t, T):
    return 0.5 * t  # nm/min

# Simulate each dopant
dopants_to_test = ['arsenic', 'phosphorus', 'boron', 'antimony']
results = {}
colors = {'arsenic': 'red', 'phosphorus': 'green', 'boron': 'blue', 'antimony': 'orange'}

for dopant in dopants_to_test:
    print(f"  Simulating {dopant}...")
    model = SegregationModel(dopant)
    
    x_final, C_final, interface_history = model.coupled_solve(
        C0.copy(), x, T=1000, t_total=60,
        oxidation_model=oxide_model,
        diffusion_solver=solver,
        dt=1.0
    )
    
    results[dopant] = {
        'x': x_final,
        'C': C_final,
        'interface': interface_history,
        'k': model.k
    }

# Plot comparison
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(14, 12))

# Concentration profiles (linear)
for dopant in dopants_to_test:
    ax1.plot(results[dopant]['x'], results[dopant]['C'], 
            color=colors[dopant], linewidth=2, 
            label=f"{dopant.capitalize()} (k={results[dopant]['k']})")
ax1.axhline(1e18, color='k', linestyle='--', alpha=0.3, label='Initial')
ax1.set_xlabel('Depth (nm)')
ax1.set_ylabel('Concentration (cm⁻³)')
ax1.set_title('Dopant Profiles After Oxidation (Linear Scale)')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, 200)

# Concentration profiles (log)
for dopant in dopants_to_test:
    ax2.semilogy(results[dopant]['x'], results[dopant]['C'], 
                color=colors[dopant], linewidth=2, label=dopant.capitalize())
ax2.axhline(1e18, color='k', linestyle='--', alpha=0.3, label='Initial')
ax2.set_xlabel('Depth (nm)')
ax2.set_ylabel('Concentration (cm⁻³)')
ax2.set_title('Dopant Profiles (Log Scale)')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0, 200)
ax2.set_ylim(5e17, 5e19)

# Pile-up ratios
pile_up_ratios = {}
for dopant in dopants_to_test:
    C_max = results[dopant]['C'].max()
    pile_up_ratios[dopant] = C_max / 1e18

ax3.bar(dopants_to_test, 
       [pile_up_ratios[d] for d in dopants_to_test],
       color=[colors[d] for d in dopants_to_test],
       alpha=0.7)
ax3.axhline(1.0, color='k', linestyle='--', alpha=0.5, label='No pile-up')
ax3.set_ylabel('Peak Concentration / Initial')
ax3.set_title('Pile-Up Ratios by Dopant')
ax3.legend()
ax3.grid(True, alpha=0.3, axis='y')

# Segregation coefficient vs pile-up
k_vals = [results[d]['k'] for d in dopants_to_test]
puf_vals = [pile_up_ratios[d] for d in dopants_to_test]
ax4.scatter(k_vals, puf_vals, s=200, 
           c=[colors[d] for d in dopants_to_test], alpha=0.7)
for i, dopant in enumerate(dopants_to_test):
    ax4.annotate(dopant.capitalize(), (k_vals[i], puf_vals[i]),
                xytext=(5, 5), textcoords='offset points', fontsize=10)
ax4.set_xlabel('Segregation Coefficient k')
ax4.set_ylabel('Pile-Up Factor')
ax4.set_title('Segregation Coefficient vs Pile-Up Strength\nLower k → Stronger Pile-Up')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nPile-Up Ratios:")
for dopant in dopants_to_test:
    ratio = pile_up_ratios[dopant]
    k = results[dopant]['k']
    print(f"  {dopant.capitalize():12s} (k={k:.2f}): {ratio:.2f}x")

print("\n✓ Lower segregation coefficient → Stronger pile-up")

## 5. Moving Interface Tracking

The Si/SiO₂ interface moves inward as oxidation proceeds.

**Volume Relations:**
- 1 volume Si → 2.2 volumes SiO₂
- For 44 nm oxide → consume 20 nm Si
- Interface moves slower than oxide grows

In [ ]:
# Track interface motion
print("Tracking interface motion during oxidation...")

# Long oxidation to see clear motion
solver = Fick1D(x_max=500, dx=1.0)
x = solver.x
C0 = np.full(len(x), 1e18)

def oxide_model(t, T):
    # Faster growth for visibility
    return 1.0 * t  # 1 nm/min

model = SegregationModel("arsenic")
x_final, C_final, interface_history = model.coupled_solve(
    C0, x, T=1000, t_total=100,
    oxidation_model=oxide_model,
    diffusion_solver=solver,
    dt=1.0
)

# Plot interface motion
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 5))

# Interface position vs time
time_array = np.array(model.time_history)
ax1.plot(time_array, interface_history, 'b-', linewidth=2, label='Interface position')
# Oxide thickness
oxide_thickness = np.array([oxide_model(t, 1000) for t in time_array])
ax1.plot(time_array, oxide_thickness, 'r--', linewidth=2, label='Oxide thickness', alpha=0.7)
ax1.set_xlabel('Time (minutes)')
ax1.set_ylabel('Distance (nm)')
ax1.set_title('Interface Motion During Oxidation')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Velocity
if len(time_array) > 1:
    dt = np.diff(time_array)
    dx_interface = np.diff(interface_history)
    v_interface = dx_interface / dt  # nm/min
    
    dx_oxide = np.diff(oxide_thickness)
    v_oxide = dx_oxide / dt  # nm/min
    
    time_mid = (time_array[:-1] + time_array[1:]) / 2
    
    ax2.plot(time_mid, v_interface, 'b-', linewidth=2, label='Interface velocity')
    ax2.plot(time_mid, v_oxide, 'r--', linewidth=2, label='Oxide growth rate', alpha=0.7)
    ax2.set_xlabel('Time (minutes)')
    ax2.set_ylabel('Velocity (nm/min)')
    ax2.set_title('Growth Rates')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

# Ratio
if len(time_array) > 1:
    ratio = interface_history / oxide_thickness
    ax3.plot(time_array, ratio, 'g-', linewidth=2)
    ax3.axhline(1/2.2, color='k', linestyle='--', alpha=0.5, 
               label='Expected ratio (1/2.2 ≈ 0.45)')
    ax3.set_xlabel('Time (minutes)')
    ax3.set_ylabel('Interface Position / Oxide Thickness')
    ax3.set_title('Position Ratio\n(Si Consumed / Oxide Grown)')
    ax3.legend()
    ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

final_oxide = oxide_thickness[-1]
final_interface = interface_history[-1]
ratio_final = final_interface / final_oxide

print(f"\nFinal State (100 minutes):")
print(f"  Oxide thickness: {final_oxide:.1f} nm")
print(f"  Si consumed: {final_interface:.1f} nm")
print(f"  Ratio: {ratio_final:.3f} (expected ~0.45)")
print(f"\n✓ Interface moves at ~45% of oxide growth rate (volume ratio effect)")

## 6. Mass Conservation Analysis

In coupled problems, total dopant may not be perfectly conserved:
- Some dopant enters oxide (lost from Si)
- Loss fraction depends on k
- Lower k → less loss

In [ ]:
# Analyze mass conservation for different dopants
print("Analyzing mass conservation for different dopants...")

solver = Fick1D(x_max=400, dx=1.0)
x = solver.x
C0 = np.full(len(x), 1e18)

def oxide_model(t, T):
    return 0.8 * t  # nm/min

mass_results = {}

for dopant in ['arsenic', 'phosphorus', 'boron', 'antimony']:
    model = SegregationModel(dopant)
    
    x_final, C_final, interface_history = model.coupled_solve(
        C0.copy(), x, T=1000, t_total=60,
        oxidation_model=oxide_model,
        diffusion_solver=solver,
        dt=1.0
    )
    
    # Check mass balance
    is_conserved, rel_error = model.mass_balance_check(
        C0, C_final, x, tolerance=0.3
    )
    
    mass_results[dopant] = {
        'k': model.k,
        'error': rel_error,
        'conserved': is_conserved
    }

# Plot results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Mass conservation error vs k
k_vals = [mass_results[d]['k'] for d in mass_results]
errors = [mass_results[d]['error'] for d in mass_results]

ax1.scatter(k_vals, errors, s=200, c=[colors[d] for d in mass_results], alpha=0.7)
for dopant in mass_results:
    ax1.annotate(dopant.capitalize(), 
                (mass_results[dopant]['k'], mass_results[dopant]['error']),
                xytext=(5, 5), textcoords='offset points')
ax1.axhline(0.3, color='r', linestyle='--', alpha=0.5, label='30% tolerance')
ax1.set_xlabel('Segregation Coefficient k')
ax1.set_ylabel('Relative Mass Loss')
ax1.set_title('Mass Conservation vs Segregation Coefficient\nHigher k → More dopant enters oxide')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Bar chart of mass loss
dopants_sorted = sorted(mass_results.keys(), key=lambda d: mass_results[d]['error'])
errors_sorted = [mass_results[d]['error'] for d in dopants_sorted]
colors_sorted = [colors[d] for d in dopants_sorted]

bars = ax2.bar(range(len(dopants_sorted)), errors_sorted, 
              color=colors_sorted, alpha=0.7)
ax2.set_xticks(range(len(dopants_sorted)))
ax2.set_xticklabels([d.capitalize() for d in dopants_sorted])
ax2.set_ylabel('Relative Mass Loss')
ax2.set_title('Dopant Loss to Oxide During Oxidation')
ax2.axhline(0.3, color='r', linestyle='--', alpha=0.5, label='30% tolerance')
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

# Add percentage labels
for i, bar in enumerate(bars):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
            f'{height*100:.1f}%',
            ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("\nMass Conservation Results:")
for dopant in sorted(mass_results.keys(), key=lambda d: mass_results[d]['k']):
    result = mass_results[dopant]
    status = "✓" if result['conserved'] else "✗"
    print(f"  {status} {dopant.capitalize():12s} (k={result['k']:.2f}): "
          f"{result['error']*100:5.1f}% loss")

print("\n✓ Lower k means more dopant stays in Si (less enters oxide)")

## 7. Practical Implications

### Device Design Considerations

1. **Arsenic Source/Drain Junctions**
   - Strong pile-up can create high-field regions
   - May increase leakage current
   - Need to account for in process design

2. **Boron Wells**
   - Moderate segregation easier to control
   - Enhanced diffusion in wet O₂ can cause depletion
   - Important for well profile control

3. **Process Integration**
   - Oxidation after implant affects final profile
   - Must simulate coupled physics for accurate prediction
   - Cannot use simple diffusion alone

### Key Takeaways

- Segregation coefficient k determines pile-up strength
- Interface motion must be tracked for accurate modeling
- Mass conservation provides important validation
- Different dopants show dramatically different behavior

## Summary

### What We Implemented (Session 5)

✅ **Segregation Model**
- Boundary condition application
- Interface velocity calculation
- Pile-up factor estimation
- Mass balance verification

✅ **Moving Boundary Tracking**
- Interface position tracking
- Grid remapping
- Volume ratio handling

✅ **Coupled Solver**
- Oxidation + diffusion integration
- Segregation at moving interface
- Time-stepping with both physics

✅ **Physical Validation**
- Correct segregation behavior
- Expected pile-up ratios
- Mass approximately conserved

### Next Steps

**Session 6-7:** Statistical Process Control (SPC)
- Western Electric rules
- CUSUM and EWMA
- Change-point detection

**Session 8-9:** Virtual Metrology & ML
- Feature engineering from FDC
- Predictive models
- Parameter calibration